In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, embedding_size):
        super(SelfAttention, self).__init__()
        self.embedding_size = embedding_size
        self.w_q = nn.Linear(embedding_size, embedding_size, bias = False)
        self.w_k = nn.Linear(embedding_size, embedding_size, bias = False)
        self.w_v = nn.Linear(embedding_size, embedding_size, bias = False)
        self.fc_out = nn.Linear(embedding_size, embedding_size, bias = False)
    
    def forward(self, query, key, value):
        query = self.w_q(query)
        key = self.w_k(key)
        value = self.w_v(value)

        attention_scores = torch.matmul(query, key.transpose(-1, -2)) * (self.embedding_size ** - 0.5)
        attention_scores = F.softmax(attention_scores, dim = -1)
        attention_out = torch.matmul(attention_scores, value)

        attention_out = self.fc_out(attention_out)
        return attention_out


In [4]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embedding_size, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embedding_size = embedding_size
        self.num_heads = num_heads
        self.head_dims = self.embedding_size // self.num_heads

        assert (self.num_heads * self.head_dims == self.embedding_size), \
            "embedding_size should be divisible by num_heads"
        
        self.w_q = nn.Linear(self.head_dims, self.head_dims, bias = False)
        self.w_k = nn.Linear(self.head_dims, self.head_dims, bias = False)
        self.w_v = nn.Linear(self.head_dims, self.head_dims, bias = False)
        self.fc_out = nn.Linear(self.head_dims, self.head_dims, bias = False)

    def forward(self, query, key, value):
        batch_size = query.shape[0]
        sequence_length_q, sequence_length_k, sequence_length_v = query.shape[1], key.shape[1], value.shape[1]

        query = query.reshape(batch_size, sequence_length_q, self.num_heads, self.head_dims)
        key = key.reshape(batch_size, sequence_length_k, self.num_heads, self.head_dims)
        value = value.reshape(batch_size, sequence_length_v, self.num_heads, self.head_dims)

        attention_scores = torch.matmul(query, key.transpose(-1, -2)) * (self.embedding_size ** -0.5)
        attention_scores = F.softmax(attention_scores, dim = -1)
        attention_out = torch.matmul(attention_scores, value)

        attention_out = attention_out.reshape(batch_size, sequence_length_q, self.num_heads*self.head_dims)
        return attention_out


In [6]:
batch_size, sequence_length, embedding_size = 1, 77, 768

query = torch.rand(batch_size, sequence_length, embedding_size)
key = torch.rand(batch_size, sequence_length, embedding_size)
value = torch.rand(batch_size, sequence_length, embedding_size)

self_attention = SelfAttention(embedding_size)
result = self_attention(query, key, value)
print(result.shape)

torch.Size([1, 77, 768])


In [5]:
batch_size, sequence_length, embedding_size = 1, 77, 768
num_head = 6
query = torch.rand([batch_size, sequence_length, embedding_size])
key = torch.rand([batch_size, sequence_length, embedding_size])
value = torch.rand([batch_size, sequence_length, embedding_size])
multi_head_self_attention = MultiHeadSelfAttention(embedding_size, num_head)
attention_out = multi_head_self_attention(query, key, value)

#### 关于self-attention的一些常见的问题?

##### Q1: self-attention的公式中, q和k.T相乘后, 为什么需要除以一个数？

1. 在自注意力的计算公式中, Q和K的转置相乘后会生成一个注意力分数的矩阵, 在这个矩阵中每个元素都代表者不同位置之间的相似性。因为我们接下来会在利用一个softmax来生成0-1之间的注意力权重。但是我们观察softmax的图像后,我们发现,如果x的值太大或者太小时,y的值会接近0或者1,这样的话就会产生梯度消失的现象,从而影响了模型的学习效果。
所以, 我们需要一个对得到的矩阵进行一个缩放的处理, 从而到X的值相对集中一些。

##### Q2：那么,为什么一定是除以根号dk呢？为什么不是除以dk呢？

1. 我们首先假设Q和K和服从标准正态分布的,也就是它们的均值为0,方差为1。那么Q和dk的点积的期望值是0,方差为dk。如果我们是除dk的话, 则按照方差的计算公式, 方差为变为dk分支1, 这就使得结果比较小, 甚至接近于0。那么softmax输出的注意力权重就会比较平均, 这并不能突出重要的关系。
    所以我们可以除以根号dk, 这样的话, 最后的方差就会变为1。从而使得softmax输出的结果比较稳定, 梯度也比较适中, 有利于反向传播中的梯度更新。

#### Q3:multi-head self-attention的作用是什么？

1. 使用了多头自注意力机制,可以丰富表达能力。单头自注意力机制在每个时刻只能基于一个权重分布来计算注意力,这可能限制了模型捕捉不同类型依赖关系的能力。多头自注意力机制通过使用多个独立的注意力头,每个头都可以学习到不同的权重分布,从而捕捉不同位置之间的多种依赖关系。
2. 多头自注意力机制可以从多个角度,多个视角来理解输入序列,从而提高模型的表达能力。
3. 多头自注意力机制可以通过并行的方式来加速计算过程,提升了效率。

##### Q4: attention-mask是干嘛的？

1. 首先在自注意力机制中, attention-mask的作用主要是为了控制模型在计算自注意力时, 对某些位置的特征进行关注和忽略。
2. attention-mask是一个矩阵, 用于在注意力计算中掩盖掉某些位置。通过, mask中的掩码位置值为一个非常大的负数, 以确保这些位置在softmax后接近0。
3. 我们可以拿swin-transformer中的shift-windows-multihead-selfattention来说, 当我们在一个窗口内计算自注意力分数的时候, 此时并不是要拿Q和所有的K相乘,所以
我们可以先拿Q和K的转置相乘, 然后创建一个mask矩阵, 其中有用的部分我们设置为0, 没有用的地方我们设置为-100, 然后和之前的attention矩阵相加, 然后在经过一个softmax
函数, 此时, -100的值所在的区域会被计算为很小的数字, 就达到了屏蔽这个区域的效果。

##### Q5: 在计算attention-scores如何对padding进行mask操作？

##### Q6：为什么在进行multi-head-self-attention的时候需要对head进行降低维度？

1. 计算效率。对每个head进行降维, 每个head的计算量变小, 整体计算成本也会减少。这会让多头注意力机制更有优势。
2. 丰富特征表达。多头自注意力机制的核心思想是通过多个head来捕捉到输入序列中不同子空间的信息。如果每个head都具有全维度, 它们学习到的特征就会高度相似, 无法有效捕捉到多元化的信息。
3. 全维度的head会带来更多的参数, 增加了模型过拟合的风险。降低维度后, 模型参数减少, 有助于降低模型过拟合的风险。

##### Q7：transformer的并行优化体现在哪些地方？

##### Q8：为什么在transformer中使用layernorm而不是batchnorm呢？

1. layernorm和batchnorm都是深度学习中让数据的分布更为集中, 从而加速模型收敛的一种方式。batchnorm在计算均值和方差的时候,是在batch维度上进行归一化的。也就是batchnorm把同一个batch内相同通道内的所有特征当作一个分布, 这是因为不同图片的相同通道是可以比较的。但是与batchnorm不同的是, 在序列数据（例如自然语言处理任务中）, 序列中的每个位置往往有很强的依赖性。考虑两个句子, “我想吃苹果”和“我买了一个苹果手机”。在这两个句子中, 苹果这一个词汇的语境是由上下文决定的。因此, layernorm是在特征维度上进行归一化的,它能够保留序列中每个位置的依赖关系, 从而更适合处理序列数据。
而transformer主要是用来处理序列数据的, 所以一般是使用layernorm, 而不是batchnorm。

2. batchnorm需要在训练和推理的时候需要计算不同的统计量（训练的时候需要使用batch内的均值和方差, 而推理的时候需要用整个数据集的均值和方差）, 而layernorm在训练和推理的时候使用i相同的均值和方差, 不需要区分训练和推理阶段, 简化了模型的实现和使用。

##### Q9：transformer计算attention的时候为什么选择点乘而不是加法？两者在计算复杂度上有什么区别？